# Customer Journey Analytics in E-commerce
Mapping the customer decision journey has become crusial tool to help companies to win customers. According to a global survey of over 7,000 marketers and e-commerce professionals, 24% of respondents ranked the customer experience as the single most exciting opportunity ahead. And, 78% agree that the customer experience creates brand differentiation and competitive advantage.

The sucessfull journey mapping requires healthy balance of qualitative and quantitative information on which organization can act uppon on. While qualitative information is gathered based on experience of customer facing fuctions, the role of data scientist comes handy when supporting marketing with quantitative knowledge.

## 1. Business Understanding
The purpose of this project is to share with fellow data scientists how they can support marketing and sales with quantitative analysis during customer journey mapping. What kind of ML algoritm to use to asnwer different marketing & sales questions like:
 1. How many buyer personas do we have?
 2. What are their unique characterstics?
 3. What is the flow in their journey?
 4. What are their main painpoints?
 5. How we can increase their engagment to our brand?
 
### 1.1 Buyer Personas
A buyer persona is a research-based representation of customer. It includes 
 - demographics, such as age, gender, or geography
 - attitudes
 - behaviors
 - motivations 
 
It describes who the customer is, their goals, their concerns, and how they think. It includes the decision criteria needed to addressed to win their business, such as how they 
 - receive information
 - weigh and evaluate different options
 - when and where they decide to buy 
 
So, basically a buyer persona is a profile of the customers who need to be engaged throughout their decision journey to drive their loyalty and advocacy as they choose comany brand over the competition. Buyer personas help better understand customers so company can choose the right marketing touchpoints and messaging, and this will attract more social shares and increase brand rankings in search engines

### 1.2 Customer Journey
A customer ourney shows the whole customer experience from initial brand awareness through purchase, and importantly after the purchase to reach customers at the moments that most influence their decisions.

These insights reveal which marketing touchpoints to use, such as search, a website, online banners, or a magazine ad, and guide what kind of messaging to communicate in each of those touchpoints to engage your customers. In that way resources an be focused in the right places and right information can be shared to customers.

As scope is quite big I will narrow it from intial brand awareness to purchase.

### 1.3 E-commerce Use Case
As most of the companies keeps data related to customer journey confidential, the use case will be demostrated on [Google Analytics Sample Dataset](https://support.google.com/analytics/answer/7586738?hl=en)

The sample dataset contains obfuscated Google Analytics 360 data from the [Google Merchandise Store](https://shop.googlemerchandisestore.com/), a real ecommerce store. The Google Merchandise Store sells Google branded merchandise. The data is typical of what you would see for an ecommerce website. It includes the following kinds of information:

 - __Traffic source__ data: information about where website visitors originate. This includes data about organic traffic, paid search traffic, display traffic, etc.
 - __Content__ data: information about the behavior of users on the site. This includes the URLs of pages that visitors look at, how they interact with content, etc.
 - __Transactional__ data: information about the transactions that occur on the Google Merchandise Store website.

In [ ]:
# IMPORTS
# -------

# Standard libraries
from importlib import reload
import pathlib
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import itertools
import ipdb
import string
import re

# 3rd party imports
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

import nltk
# nltk.download(['wordnet', 'stopwords'])
STOPWORDS = nltk.corpus.stopwords.words('english')

from scipy import stats
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster

from statsmodels.formula.api import ols

import scikit_posthocs as sp

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import f1_score

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

from google.cloud import bigquery
from tqdm import tqdm_notebook as tqdm

# Local libraries
from bigquery_ import BigqueryTable
from bigquery_ import BigqueryDataset
import helper


# AUTHENTIFICATION & ACCESS
# -------------------------

# Supply your service account key
service_account = pathlib.Path(
    ('C:/Users/Fredo/Google Drive/Knowledge Center'
    '/Data Scientist Nanodegree/customer-journey-31622634430d.json')
)

# Initiate biqquery client
client = bigquery.Client.from_service_account_json(service_account.absolute())

# Authenticate service account
# Authenticate_service_account(service_account_key)

# Get dataset reference
project = 'bigquery-public-data'
dataset_id = 'google_analytics_sample'
dataset_ref = client.dataset(dataset_id, project=project)
dataset = BigqueryDataset(client, dataset_ref)

## 2. Data Understanding
Data can be accessesed via BigQuery. They are stored in form of daily tables from 1st Aug 2016 to 1st Aug 2017. As the dataset schema does not reveal any fields description, it needs to be merged with [BigQuery Export Schema](https://support.google.com/analytics/answer/3437719?hl=en).

In [ ]:
# Investigate dataset schema
display(dataset.schema.shape)
display(dataset.schema['table_name'].head(n=3))
display(dataset.schema['table_name'].tail(n=3))

In [ ]:
# Investigate table schema
# Load BigQuery export schema
bq_exp_schema = pd.read_excel('google_analytics_schema.xlsx', 
                              sheet_name='bq_exp_schema')

# Load google analytics sample table
table_id = 'ga_sessions_20170801'
table_ref = dataset_ref.table(table_id)
table = client.get_table(table_ref)

# Recast table to custom BigqueryTable class
table.__class__ = BigqueryTable

# Load google analytics schema and merge it with export schema
# to get field descriptions
ga_schema = table.schema_to_dataframe(bq_exp_schema)
table.display_schema(ga_schema)

In [ ]:
# Calculate number of variables, excluding records
ga_schema[ga_schema['Data Type'] != 'RECORD'].shape[0]

In [ ]:
# Analyse variable levels
# -----------------------
# Running this code will take 18h on Intel(R) i5 2.2GHz, RAM 8GB,

# Update schema by variable level characteristics
# ga_schema = dataset.get_levels(client, bq_exp_schema)

# Save updated schema with levels to HDF5 file
# ga_schema.to_hdf('temp_data.h5', 'schema', mode='w', table=True)

In [ ]:
# load updated schema with levels from HDF5 file
ga_schema = pd.read_hdf('temp_data.h5', 'schema')

# Print variable levels overview
schema_levels = (ga_schema
                 .groupby(['Num of Levels'])
                 .agg({'Field Name': 'count', 
                       'Levels': lambda level: set().union(*level.to_list())})
                 .rename({'Field Name': 'Num of Variables'})
                 .sort_index())
table.display_schema(schema_levels[:2])

In [ ]:
# Select variables and analyze scales
# ----------------------------------

# Discard variables with 1 level
schema_multi_levels = (
    ga_schema[(ga_schema['Num of Levels'] > 1) 
              | (ga_schema['Field Name'] == 'totals.visits')]
    .set_index('Field Name')
)
table.display_schema(schema_multi_levels.drop(columns=['Levels']))

In [ ]:
# Discard not related or too detailed and too general variables 
# (done manually in excell file)
schema = (pd.read_excel('google_analytics_schema.xlsx', 
                         sheet_name='ga_schema_uni_level_excl')
          .set_index('Variable Name'))
display(schema)

In [ ]:
# Variable selection, Scale and personas characteristics summary
for column in ['Status', 'Scale', 'Field Group']:
    display(schema.groupby(column)[column].count())

In [ ]:
# Only selected and considered variables summary
for column in ['Status', 'Scale', 'Field Group']:
    display(schema[(schema['Status'] == 'CONSIDER')
                   | (schema['Status'] == 'SELECTED')]
            .groupby(column)[column]
            .count())

### 2.1 Data Understanding Discussion
There are 366 tables in the dataset. Each table represent one day from 1 Aug 2016 to 1 Aug 2017. Each table row represents one [session](https://support.google.com/analytics/answer/2731565?hl=en) and includes [nested and repeated fields](https://www.kaggle.com/alexisbcook/nested-and-repeated-data), which expands granularity of data to following levels
- `session`
    - `customDimesions`
    - `hits`
        - `customDimensions`
        - `products`
            - `customDimensions`
            - `customMetrics`
        - `promotions`
        - `experiments`
        - `customVariables`
        - `customDimensions`
        - `customMetrics`
        - `publisher_info`

Totaly there is 306 variables where 182 have only 1 level/code. These are missing data columns, columns not available in demo dataset or constant columns not giving value to analysis objectives. All of them will be discarded except of `totals.visits` for aggregation purposes.

Further more 65 Variables has been excluded for following reasons
- not related for characteristics of buyer personas
- too detailed considering practical description of buyer personas
- too general not giving enough differenciation of buyer personas

Remaining 60 variables were split to two categories
- 34 selected just could discribe personas and give practical meaningful information
- 26 considered to be used later in case personas description is not detial enough

Out of 60 variables 40 are nominal, 12 numeric, 3 binary, 3 ordinal, 1 datetime stamp, 1 day stamp. Out of 60 48 are behavior, 9 demographics and 3 attitude.

## 3. Data Preparation
To help marketing team to answer how many buyer personas are there and what are their characterstics, I will develope taxonomy of the customers based on their purchase behavior such as:
 - How much do they spend?
 - How frequently do they buy?
 - When they made first purchase?
 - When they made most recent purchase?

them I will look to other customer characteristics and create profile for each taxonomy group (segment, buyer persona).

The primary objective is to develope a taxonomy that segments customers into groups with similar purhase behaviors. Ones identified, marketing strategies with different appeals can be fomulated for each group. The customer segmenation method used is called [RFM](https://link.springer.com/article/10.1057/dbm.2012.17) and following transaction data are needed:
 - time
 - transaction_id
 - transaction_revenue
 - client_id

to be aggregated on customer level and calculate:
 - `first_purchase`: days after 1st customer purchase
 - `recency`: days after last customer purchase
 - `frequency`: # of the customer purchases
 - `monetary`: # total revenue of customer purchases
 
From ML perspective clustering algorithm will be used. Outliers and multicoliearity need to be checked as clustering solutions are sensitive to both.

In [ ]:
%%time
# Query selected variables from google analytics dataset
query = '''
SELECT
    date,
    hits.transaction.transactionId AS transaction_id,
    fullVisitorId AS client_id,
    (hits.transaction.transactionRevenue / 1e6) AS revenue
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`,
    UNNEST(hits) AS hits
WHERE
    _TABLE_SUFFIX BETWEEN @start_date AND @end_date
    AND hits.transaction.transactionRevenue IS NOT NULL
ORDER BY
    date
'''

query_params = [
    bigquery.ScalarQueryParameter(
        'start_date', 'STRING', 
        dataset.schema['table_name'].values[0].split('_')[-1]),
    bigquery.ScalarQueryParameter(
        'end_date', 'STRING', dataset.schema['table_name'].values[-2].split('_')[-1])
]

job_config = bigquery.QueryJobConfig()
job_config.query_parameters = query_params
df = client.query(query, job_config=job_config).to_dataframe()

# Cast variables
df['date'] = pd.to_datetime(df['date'])
df['client_id'] = df['client_id'].astype(str)

# Prints dataframe charactersitics
display(df.head())
display(df.info())
display(df.describe(include=np.number))
display(df.describe(exclude=np.number))

In [ ]:
# Calcuate RFM variables
present = df['date'].max()
rfm = df.groupby('client_id').agg({
    'date': {'first_purchase':lambda date: (present - date.min()).days,
             'recency': lambda date: (present - date.max()).days},
    'transaction_id': {'frequency':  lambda id_: len(id_)},
    'revenue': {'monetary': lambda revenue: revenue.sum()}})
rfm.columns = rfm.columns.droplevel()

display(rfm.head())
display(rfm.describe())

In [ ]:
# Plot distributions log scale on y-axis of diagonal historgrams
sns.pairplot(rfm); # diag_kws=dict(log=True)

In [ ]:
# Apply log transformation to exponential and chi-squared distributions
rfm[['ln_frequency', 'ln_monetary']] = np.log1p(rfm[['frequency', 'monetary']])
sns.pairplot(rfm[['frequency', 'ln_frequency', 'monetary', 'ln_monetary']],
             diag_kws=dict(log=True));

In [ ]:
# Investigate multicolinearity
sns.heatmap(rfm.corr(), annot=True);

In [ ]:
# Investigate oultiers based on multivariate dissimilarity
# --------------------------------------------------------

# Create original and transformed variable sets
idxs = {'original': pd.Index(['first_purchase', 'recency', 
                              'frequency', 'monetary']),
        'transformed': pd.Index(['first_purchase', 'recency', 
                                 'ln_frequency', 'ln_monetary'])}

# Calculate dissimilarity for both sets
dissimilarity = pd.DataFrame()
for idx_name, idx in idxs.items():
    dissimilarity[idx_name] = helper.get_dissimilarity(rfm[idx])

# Print distributions of dissimilarities
axes = dissimilarity.hist(log=True)
plt.suptitle('Distribution of Customers Dissimilarity')
for ax in axes[0]:
    ax.set_xlabel('Dissimilarity')
axes[0, 0].set_ylabel('Log # of customers')
plt.show()

# Display overview
ext_summary = (pd.concat([rfm[idxs['original']], dissimilarity], 
                             axis=1)
                 .sort_values('transformed', ascending=False))
display(ext_summary[:10])
display(dissimilarity.describe())

In [ ]:
# Monetary value of outliers
display((ext_summary['monetary'][1].sum() / rfm['monetary'].sum()).round(3))

### 3.1 Data Preparation Discussion
There are 12028 transactions, 9962 customers and no missing values in dataset. 

It seems that google merchendise store:
- is slightly growing new customers from serious drop-out of 200 days ago (present means 1st Aug 2017)
- was doing extremely well in number of customers purchasing goods 220-240 days ago
- but is not doing well keeping customers as >75% of them purchased only ones

#### 3.1.1 Multicollinearity
`first_purhase` and `recency` are higly collinear which inflates importance of time dimension over frequency and monetary value and can negatively effects forming of clusters. I decided to exclude `first_purchase` from analysis.

#### 3.1.2 Distributions
`first_purchase` and `recency` are uniformly or multimodal distributed.`frequency` approximate expenential and `monetary` chi-square distribution. As these are highly skewed distributions both of variables are transformed using natural logarithm: $x_{tr} = ln(x+1)$. Transformation will reduce number of outliers and enable clustering algorithm to form more homogenous clusters with more equal sizes.

#### 3.1.3 Outliers
Multivariate dissimilarity has been used to identify outlier customers from average customer. While raw data shows 1 outlier with huge customer monetary value of 128K$. Even if this is outlier it need to be kept in dataset as it represents 7.2% of total revenue.

Using transformation shows no outliers but 8 underpresented customers in the population, which need to be kept in dataset.

## 4. Modeling
As there is no conceptual knowledge of number of the buyer personas, I will hold to practical consideration of having 3-7 clusters, suppose that distict marketing strategy will be developed for each of the buyer persona. First hierarchical clustering will be used in order to examine all possible cluster solutions. The candidate solutions will be selected for non-hierachical algorithm to form more compact clusters. Then final solution from candidates will be selected in respect to practical consideration.

### 4.1 Research design of the cluster analysis

#### 4.1.1 Define similarity measure
Given that all tree clustering variable are metric the __Euclidean Distance__ is chosen as the similarity measure. If `first_purchase` would need to be included from business point of view, than __Mahalanobis Distance__ could be used to deal with multicolinearity issue.

#### 4.1.2  Sample size
As customer sample is quite big (9962 customers) we can split sample by half to cross-validate cluster solution. Random split can be used with conditions that underepresented customers will remain in both samples to keep data structure. 

#### 4.1.3  Standardization
As the averages and standard deviations of the variables vary a lot, they need to be standardized before entering to cluster algorithm in order to ensure assigning equal weights to each variable when forming clusters.

### 4.2 Assumptions in cluster analysis
#### 4.2.1 Sample representativeness
Considering objective of identifying buyer personas, whole customer population (population = all customer transactions during existance of Google Merchendise Shop) or random sample from that population should be taken. Unfortunately this was not done as available sample is related to fixed time period from 1 Aug 2016 to 1 Aug 2017. For the purpose of this excercise I will limit populaton to this time period. When population is randomly split to half, both of samples can be considered representative.

#### 4.2.2 Multicollinearity
Multicollinearity effects are minimized trough the variable selection process, where `first_purchase` variable was excluded from dataset.

### 4.3 Selecting clustering algorithm
- __Aglomerative Clustering__ will be used in the first step, due to its ability to generate all clustering solutions, time efficiency to handle big samples. The __ward__ method will be used based on __Squared Eclidian Distance__ with advantage of forming homogenous clusters with relatively equal in size. This is especially efficient to used dataset where clustering variable distributions are hihgly skewed with outliers.
- __K-Nearest Neighbour__ non-hierarchical algorithm will be used, due to its ability to handle large samples especialy __MiniBatch__ version  in second step to form more compact clusters

### 4.3.1 Hierarchical Clustering

In [ ]:
# Randomly split data into two sets, and keep outliers in each set
# -----------------------------------------------------------------

# Make results deterministic
random_state=1

# Select variables
X = rfm.drop(columns=['first_purchase'])
for idx_name, idx in idxs.items():
    idxs[idx_name] = idx.drop('first_purchase', errors='ignore')

# Split datasets
threshold = 10
X_names = ['train', 'test']
size = 0.5
Xs = helper.split_data(X, idxs['transformed'], X_names, threshold, 
                       test_size=size, random_state=random_state)

for X_name, X in Xs.items():
    print('Dataset', X_name, 'of shape', X.shape)

In [ ]:
# BUILD AND OPTIMIZE MODEL FOR 1st DATASET
# ----------------------------------------

# Standardize datasets
scalers={}
Xs_std = {}
idxs['all_inputs'] = pd.Index([
    'recency', 'frequency', 'monetary', 'ln_frequency', 'ln_monetary', 
    'recency_std', 'frequency_std', 'monetary_std','ln_frequency_std', 
    'ln_monetary_std'
])

for X_name, X in Xs.items():
    scalers[X_name] = StandardScaler().fit(X)
    Xs_std[X_name] = scalers['train'].transform(X)
    Xs_std[X_name] = pd.DataFrame(Xs_std[X_name], 
                                  columns=Xs[X_name].columns+'_std', 
                                  index=Xs[X_name].index)
    Xs[X_name] = (pd.concat([Xs[X_name], Xs_std[X_name]], axis=1)
                  .reindex(idxs['all_inputs'], axis=1))

# Agglomerative Clustering with average linkage
idxs['transformed_std'] = pd.Index(
    ['recency_std', 'ln_frequency_std', 'ln_monetary_std'])

agl_cluster = linkage(Xs_std['train'][idxs['transformed_std']], 'ward')

# Construct dendrogram
plt.figure(figsize=(6, 6))
dn = dendrogram(agl_cluster, p=30, truncate_mode='lastp', orientation='right')
plt.title('Customers Dendrogram')
plt.xlabel('Aglomeration Coeficient')
plt.ylabel('Customers')
plt.show()

In [ ]:
# Step 1: Inspect outliers in agglomeration schedule if any
pass

# Step 2: Consider removing outliers if appropriate
pass

# Step 3: Run algomerative clustering again go to step 1
pass

# Note: no outliers

In [ ]:
# Determine number of candidate solutions
# ---------------------------------------

# Get aglomeration shedule
agl_schedule = pd.DataFrame(
    agl_cluster,
    columns=['cluster1', 'cluster2', 'coefficient', 'cluster_size'],
    index=pd.Index(range(1, Xs_std['train'].shape[0]), name='stage')
)

# Calculate proportional heterogeneity increase
agl_schedule['num_of_clusters'] = list(reversed(agl_schedule.index))
agl_schedule['prop_heterogeneity_increase'] = (
    agl_schedule['coefficient']
    .diff()
    .shift(periods=-1)
    / agl_schedule['coefficient']
)

# Print last x stages of algomeration schedule
stages = 15
display(agl_schedule[-stages:])

In [ ]:
# Print heterogenity scree plot
(agl_schedule
 .set_index('num_of_clusters')
 ['prop_heterogeneity_increase'][-stages:]
 .plot(title='Heterogeneity Scree Plot'))
plt.ylabel('Proporion heterogeneity increase to next stage');

In [ ]:
# Selected candidate solutions
num_seeds = [3, 6]

# Profile candiate solutions
# --------------------------

for k in num_seeds:
    
    # Fenerate candidate solution labels
    labels = fcluster(agl_cluster, k, criterion='maxclust')
    Xs['train']['hierarchic_clusters_{}'.format(k)] = labels
    
    # Display cluster sizes
    display(Xs['train'].groupby('hierarchic_clusters_{}'.format(k))
                       .size()
                       .rename('cluster_sizes'))

display(Xs['train'].head())

In [ ]:
# Test candidate solutions
# ---------------------------

# Initiate profile function arguments
idxs['hierarchic_clusters'] = Xs['train'].columns[
    Xs['train'].columns.str.contains('hierarchic_clusters')]

post_hoc_test = sp.posthoc_nemenyi

# Execute cluster profile tests, print cluster profiles & scatterplots
for solution in idxs['hierarchic_clusters']:
    summary, post_hoc, prof_ax, clst_pg = (
        helper.analyze_cluster_solution(
            Xs['train'], idxs['transformed_std'], 
            solution, post_hoc_fnc=post_hoc_test
        )
    )
    
    # Print original variable means and averages per cluster
    str_ = 'Profile characteristics'
    print(str_ + '\n' + '-' * len(str_))

    display(Xs['train']
            .groupby(solution)
            [idxs['original']]
            .agg(['mean', 'median'])
            .round(1)
            .swaplevel(axis=1)
            .sort_index(level=0, axis=1))
    
    print('\n')

In [ ]:
# Revenue overview of 6 cluster solution
monetary_cls_6 = (Xs['train']
                  .groupby('hierarchic_clusters_6', as_index=False)
                  ['monetary']
                  .sum())
monetary_cls_6['prop_monetary'] = (monetary_cls_6['monetary'] 
                                   / Xs['train']['monetary'].sum())
display(monetary_cls_6)

#### 4.3.1.1 Hierarchical Clustering Discussion
##### 4.3.1.1.1 Initial Cluster Results
Dendrogram and aglomeration schedule helps to understnand the clustering process. 

Dendogram shows that indeed natural logarithm tranformation of `recency` and `monetary` and __ward method__ have helped to form relatively equal clusters with no outliers. When inspecting solution with 30 clusters the cluster size ranges from 13 to 439customers. There are no clusters with single customer which shows alorithm robustness towards outliers.

Ex. if __Average Linkage__ method would be used there would be tree clusters [4977, 4983, 4980] with one customer in later algomeration stages. It would be necessary to remove those and re-run the algorithm again to reach similar cluster homogenity and sizes as in __Ward Method__ case.

_Note:_ To try __Average Linkage__ method just change `ward` to `average` in cell In [18]

##### 4.3.1.1.2 Determining Preliminary Cluster Solutions
Candidate cluster solutions will be specified using __Stopping Rule__. It is based on assesing the changes in heterogeity between cluster solutions. The basic rational is that when large change in heterogeneity occur in moving form one stage to the next, the prior cluster solutions should be selected because the next stage is joining quite diffrent clusters. The proportion heterogeneity change to next stage is calculated as: 

$prop\_heterogeneity\_increase_{\text{i}} = \cfrac{coefficient_{\text{i+1}} - coefficient_{\text{i}}}{coefficient_{\text{i}}}$

__Heterogeneity Scree Plot__ shows that proprotionaly highest heterogeneity increase is when going from 3 to 2 and 6 to 5 clusters solutions. Therefore two solution candidates with 3 and 6 clusters were further selected for profiling.

##### 4.3.1.1.3 Profiling of Cluster Solutions
Before proceeding to nonhierarchical analysis, the profiling of selected cluster solutions is done to confirm that the differences between customer clusters are distincitive and sifnificant in recency and frequency of purchase and revenue (clustering variables)

Using __one-way ANOVA__ to identify clustering variable is not practical in this case as ANOVA prerequisites of __residuals normality__ (Shapiro-Wilk Test) and __group variance homogeneity__ (Leven's test) are not met. Instead non parametric __Kruskal-Wallis test__ was used to evaluate overal sifnificance and __Post-hoc Nemenyi__ was used to evaluate pair-wise significance.

The results shows that there are significant diffrences between the clusters on all tree variables in both solutions. These provide initial evidence that each of the tree and six clusters is distinctive.

Examination of means of cluster variables shows:
 - tree clusters solution
   - cluster 1 (609 customers) has highest mean on frequency 3.9 and monetary value 867USD representing ___"frequent shopers and high spenders"___
   - cluster 2 (2171 customers) with highest recency 271 days, low frequency 1, most probably representing ___"customer churn"___
   - cluster 3 (2204 customers) with lowest recency 94 days, low frequency 1, represening ___"new customers"___ 
   
 - six clusters solution
   - cluster 1 (83 customers) has highest mean on frequency 7.7 and monetary value 4162USD representing ___"frequent shoppers and huge spenders"___ most probably business customers
   - cluster 2 (526 customers) has second highest mean on frequency 2.3 (no significant diffrence with cluster 1 frequency mean 7.7) and second highest monetary value 347USD representing ___"frequent shoppers and moderate spenders"___
   - cluster 3 (428 customers) has highest recency 319 days, lowest frequency 1 and moderate monetary value 315USD representing ___"moderate spenders churn"___
   - cluster 4 (1743 customers) with 2nd highest recency 259 days, lowest frequency 1 and low monetary value 41USD represening ___"low spenders churn"___
   - cluster 5 (1183 customers) low monetary value 32USD, recency 71 days and frequency 1 representing ___"new low spenders"___
   - cluster 6 (1021 customers) low frequency 1 and moderate monetary value 216USD representing ___"one time moderate spenders"___
   
Both of the solutions will be considered in non-hierarchical analysis due to
 - simplicity and marketing cost efficiency of tree custers solution, where only 3 strategies need to be developed
 - ability to specify buyer personas with higher granularity, where cluster 1 typology is in center of interest representing only 83 customers who generated 35% (345K USD) of revenue.

### 4.3.2 Nonhierarchical Clustering
Nonhierarchical clustering methods have the advantage of being able to better "optimize" cluster solutions by reassigning observations until maximum homogeneity (similarity) is achieved. This second step in clustering process uses number of clusters (seeds) determined in hierarchical clustering. The clustering solutions are then compared to
- how do they generalized on population
- their predictability in respect to additional taxonomy variables (not included in this example)
- aplicability to business objective (forming marketing strategies for different buyers personas and best ROI)

#### 4.3.2.1 Seed Points Generation
Tree and six seeds will be used respectively. No random method neither custom selection of centroids will be used. There are no priori business information about rfm characteristic of buyer personas and random method suffers from local optima issue. To cope with this sklearn __K-means++__ initialization method will be used to initialize centroids to be distant from each other, leading to provably better results than random initiallization as shown in the [reference](http://ilpubs.stanford.edu:8090/778/1/2006-13.pdf).

#### 4.3.2.2 Clustering Algorithm
__Optimization algorithm__ will be used. It allows reasignment of observation among clusters until a minimum level of heterogeneity is reached. Sklearn `KMeans` object's parameter `algorithm='auto'` will determine which algorithm branch to use according data structure. `full` for sparce data and `elkan` for dense data. 

In [ ]:
# K-mean clustering for selected candidate solutions
kmeans, labels = {}, {}
for k in num_seeds:
    
    # Fit model
    kmeans['non_sorted_{}'.format(k)] = (
        KMeans(n_clusters=k, 
               random_state=random_state, 
               init='k-means++')
        .fit(Xs['train'][idxs['transformed_std']])
    )
    
    # Predict lables
    labels['non_sorted_{}'.format(k)] = (
        kmeans['non_sorted_{}'.format(k)]
        .predict(Xs['train'][idxs['transformed_std']])
    )
    
    Xs['train']['nonhierarchic_clusters_{}'.format(k)] = (
        labels['non_sorted_{}'.format(k)]+1
    )
    
# Display cluster sizes
idxs['nonhierarchic_clusters'] = Xs['train'].columns[
    Xs['train'].columns.str.contains('nonhierarchic_clusters')]

for solution in idxs['nonhierarchic_clusters']:
    display(Xs['train'].groupby(solution)
                      .size()
                      .rename('cluster_sizes')) 

In [ ]:
# Test candidate solutions
# ---------------------------

# Execute cluster profile tests, print cluster profiles & scatterplots
for solution in idxs['nonhierarchic_clusters']:
    summary, post_hoc, prof_ax, clst_pg = (
        helper.analyze_cluster_solution(
            Xs['train'], idxs['transformed_std'], 
            solution, post_hoc_fnc=post_hoc_test
        )
    )
    
    # Print original variable means and averages per cluster
    str_ = 'Profile characteristics'
    print(str_ + '\n' + '-' * len(str_))

    display(Xs['train']
            .groupby(solution)
            [idxs['original']]
            .agg(['mean', 'median'])
            .round(1)
            .swaplevel(axis=1)
            .sort_index(level=0, axis=1))
    
    print('\n')

In [ ]:
# Evaluate within clusters homogeneity and 
# between clusters heterogeneity using Silhouette coeficient
idxs['clusters'] = Xs['train'].columns[
    Xs['train'].columns.str.contains('clusters')]

score = pd.Series(index=idxs['clusters'], name='silhouette_score')

for k, solution in zip(num_seeds * 2, idxs['clusters']):
    score[solution] = silhouette_score(Xs['train'][idxs['transformed_std']],
                                       Xs['train'][solution], 
                                       random_state=random_state)
# Diplay overview
score = score.sort_values(ascending=False)
score.plot.bar(title = 'Silhouette coefficient')
plt.show()
display(score)

##### 4.3.2.1 Discussion on Nonhierarchical Clustering
Hierarchical and nonhierarchical solutions are very similary when looking to profile vizualization. The nonhierarchical solutions have ability to form more homogenous clusters with higher distictivness between clusters as it can be seen comparing clusters vizualization. This is due to the ability to reassign observations between clusters. The both 3 and 6 nonhierarchical clusters solutions have higher __Silhouette Score__ compared to their hierarchical versions.

##### 4.3.2.1.1 Profiling of Cluster Solutions
The results are very similar to hierarchical solution. They shows that there are significant diffrences between the clusters on all tree variables in both solutions. These provide initial evidence that each of the tree and six clusters is distinctive.

Examination of means of cluster variables shows sligth changes in profiles:
 - tree clusters solution
   - cluster 1 (497 customers) has highest mean on frequency 3.2 and monetary value 1129USD representing ___"frequent shopers and high spenders"___
   - cluster 2 (2298 customers) with highest recency 273 days, low frequency 1, most probably representing ___"customer churn"___
   - cluster 3 (2189 customers) with lowest recency 87 days, low frequency 1, represening ___"new customers"___ 
   
 - six clusters solution
   - cluster 5 (83 customers) has highest mean on frequency 7.7 and monetary value 4156USD representing ___"frequent shoppers and huge spenders"___ most probably business customers (matching hierarchical cluster 1)
   - cluster 3 (494 customers) has second highest mean on frequency 2.3 (no significant diffrence with cluster 1 frequency mean 7.7) and second highest monetary value 368USD representing ___"frequent shoppers and moderate spenders"___ (matching hierarchical cluster 2)
   - cluster 2 (794 customers) has highest recency 294days, lowest frequency 1 and moderate monetary value 198USD representing ___"moderate spenders churn"___ (matching hierarchical clusters 3)
   - cluster 6 (1478 customers) with 2nd highest recency 260 days, lowest frequency 1 and low monetary value 35USD represening ___"low spenders churn"___ (matching hierarchical cluster 4)
   - cluster 1 (1490 customers) has low monetary value 39USD, recency 77 days and frequency 1 representing ___"new low spenders"___ (matching hierarchical cluster 5)
   - cluster 4 (1645 customers) has low frequency 1 and moderate monetary value 310USD representing ___"one time moderate spenders"___ (matchin hierarchical cluster 6)

## 5. Evaluation
The purpose of this section is to assess if generalizability of the solution to population.

### 5.1 Clusters Stability
Factors such as initial seed points, observation ordering can effect cluster membership in nonhierarchical clustering. To check stability of the clusters, dataset is sorted by monetary value and compared with original 3 and 6 cluster solutions.
Solutions are cross tabulated for comparison of missmatched cluster membership where following rules are applied:

| Cluster Stability 	| Missmatch in % 	|
|-------------------	| ----------------	|
| very stable       	|     <0-10>     	|
| stable            	|     (10-20>    	|
| somewhat stable   	|     (20-25>    	|
| not stable        	|       >25      	|

___Note___: Clusters numbers may not correspond to each other across diffrent algorithms and differently sorted datasets. Therefore do not expect highest numbers on diagonal of confusion matrix. Matching of the clusters need to be doublechecked vissually on scatterplots.

In [ ]:
# Generate sorted dataset by monetary value
Xs['train_sorted'] = Xs['train'].sort_values('monetary_std').copy()

# K-mean clustering for selected candidate solutions
for k in num_seeds:
    
    # Fit model
    kmeans['sorted_{}'.format(k)] = (
        KMeans(n_clusters=k, 
               random_state=random_state, 
               init='k-means++')
        .fit(Xs['train_sorted'][idxs['transformed_std']])
    )
    
    # Predict lables
    labels['sorted_{}'.format(k)] = (
        kmeans['sorted_{}'.format(k)]
        .predict(Xs['train_sorted'][idxs['transformed_std']])
    )
    
    Xs['train_sorted']['nonhierarchic_sorted_clusters_{}'.format(k)] = (
        labels['sorted_{}'.format(k)] + 1
    )

    # Cross tabulation
    cross_tab, missmatch, total_missmatch = helper.get_missmatch(
        index=Xs['train']['hierarchic_clusters_{}'.format(k)],
        columns=Xs['train_sorted']['nonhierarchic_sorted_clusters_{}'.format(k)],
        rownames=['original'],
        colnames=['sorted']
    )

    print(cross_tab,'\n')
    print(missmatch, '\n')
    print('Total missmatch proportion: {:.2f}\n\n\n'.format(total_missmatch))

### 5.1.1 Clusters Stability Discussion
The 3 and 6 cluster solution are stable as membership missmatched is > 10% (11%, 16% respectively). The 6 cluster solution showed 38% missmatched. There is a possibility to improve stability of 6 cluster solution by selecting custom centerpoints, but this would need to be supported by priori bussiness information.

## 5.2 Clusters Generalizability
Clustering methods belongs to unsupervised machine learning algorithms, so there are no ground true labels to calculate accuracy to check how model generalize to population. The diffrent approach can be used using cross tabulation between two solutions and calculating missmatch where same rules above apply. The following procedure is used: 
 - Data are randomly splitted by half to `train` and `test` sample.
 - The `train` sample is used to train firt clustering model which needs to be evaluated. 
 - "True" labels are estimated by second clustering model trained on `test` sample.  
 - Predicted labels are aquired using `KMeans.predict` function of the first fitted model
 - First model generalizability is evaluated using __confusion matrix__ cross-tabulating "True" and predicted labels

In [ ]:
# Validate solution generalizability to whole population
# ------------------------------------------------------
for k in num_seeds:
    
    kmeans['train_{}'.format(k)] = kmeans['non_sorted_{}'.format(k)]
    
    # Fit model on test dataset
    kmeans['test_{}'.format(k)] = (
        KMeans(n_clusters=k, 
               random_state=random_state, 
               init='k-means++')
        .fit(Xs['test'][idxs['transformed_std']])
    )
    
    # Predict lables with model fitted on test dataset
    labels['test_true_{}'.format(k)] = (
        kmeans['test_{}'.format(k)]
        .predict(Xs['test'][idxs['transformed_std']])
    )
    
    Xs['test']['true_clusters_{}'.format(k)] = (
        labels['test_true_{}'.format(k)] + 1
    )

    # Predict lables with model fitted on train dataset
    labels['test_predicted_{}'.format(k)] = (
        kmeans['train_{}'.format(k)]
        .predict(Xs['test'][idxs['transformed_std']])
    )
    
    Xs['test']['predicted_clusters_{}'.format(k)] = (
        labels['test_predicted_{}'.format(k)] + 1
    )
    
    # Cross tabulation
    cross_tab, missmatch, total_missmatch = helper.get_missmatch(
        index=Xs['test']['true_clusters_{}'.format(k)],
        columns=Xs['test']['predicted_clusters_{}'.format(k)],
        rownames=['true'],
        colnames=['predicted']
    )

    print(cross_tab,'\n')
    print(missmatch, '\n')
    print('Total missmatch proportion: {:.2f}\n\n\n'.format(total_missmatch))

### 5.2.1 Discussion on Clusters Generalizability
The 3 cluster solution generalize very well to population with 1% clusters membership missmatch (Accuracy Error). The 6 cluster solution generalize well with 6% error. It is acceptable to profile and compare both solutions on additional non clustering variables.

In [ ]:
# Predict cluster labels for whole dataset
# ----------------------------------------

# Standardize data
X = rfm.drop(columns=['first_purchase'])
X_std = scalers['train'].transform(X)
X_std = pd.DataFrame(X_std, 
                     columns=X.columns+'_std', 
                     index=X.index)
X = (pd.concat([X, X_std], axis=1)
              .reindex(idxs['all_inputs'], axis=1))

# Predict labels
labels = {}
for k in num_seeds:
    labels[k] = (kmeans['train_{}'.format(k)]
                 .predict(X[idxs['transformed_std']]))
    
    X['clusters_{}'.format(k)] = labels[k] + 1

display(X.head())

## 5.3 Profiling Cluster Solution on Additional Variables
The profiling stage involves discribing the charateristics of the each buyer persona (cluster) to explain how it may differ in relevant dimensions as demograpics, consumption patterns and behaviors. These are the variables not included in cluster analysis. This section provides answer on second quastion: What are the unique characteristics of buyer personas?

### 5.3.1 Data Understading

In [ ]:
%%time
# Query additional variables from google analytics dataset
query = '''
SELECT
    CONCAT(fullVisitorId, CAST(visitId AS STRING)) AS session_id,
    visitNumber AS visit_number,
    date,
    totals.pageviews AS pageviews,
    totals.timeOnSite AS time_on_site,
    trafficSource.referralPath AS referral_path,
    trafficSource.campaign AS campaign,
    trafficSource.source AS source,
    trafficSource.medium AS medium,
    trafficSource.keyword AS traffic_keyword,
    trafficSource.adContent AS ad_content,
    trafficSource.adwordsClickInfo.page AS ad_page,
    trafficSource.adwordsClickInfo.slot AS ad_slot,
    trafficSource.adwordsClickInfo.adNetworkType AS ad_network_type,
    trafficSource.isTrueDirect AS direct_traffic,
    device.browser AS browser,
    device.operatingSystem AS operating_system,
    device.deviceCategory AS device_category,
    geoNetwork.continent AS continent,
    geoNetwork.subContinent AS subcontinent,
    geoNetwork.country AS country,
    geoNetwork.region AS region,
    geoNetwork.metro AS metro,
    geoNetwork.city AS city,
    geoNetwork.networkDomain AS domain,
    customDimensions.value AS sales_region,
    hits.hour AS hour,
    hits.transaction.transactionId AS transaction_id,
    hits_product.productSKU AS product_sku,
    hits_product.v2ProductName AS product_name,
    hits_product.v2ProductCategory AS product_category,
    hits_product.productBrand AS product_brand,
    (hits_product.productPrice / 1e6) AS product_price,
    hits_product.productQuantity AS product_quantity,
    hits.social.hasSocialSourceReferral AS is_social_referral,
    hits.social.socialInteractionAction AS social_action,
    hits.social.socialInteractionNetwork AS social_interaction_network,
    hits.social.socialInteractionNetworkAction AS network_action,
    hits.social.socialInteractions as social_interactions,
    hits.social.socialNetwork as social_network,
    hits.social.uniqueSocialInteractions as unique_social_interactions,
    hits.contentGroup.contentGroup1 AS product_brand_grp,
    hits.contentGroup.contentGroup2 AS product_category_grp,
    fullVisitorId AS client_id,
    channelGrouping AS channel_group
FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    LEFT JOIN UNNEST(customDimensions) AS customDimensions
    LEFT JOIN UNNEST(hits) AS hits
    LEFT JOIN UNNEST(hits.product) AS hits_product
WHERE
    _TABLE_SUFFIX BETWEEN @start_date AND @end_date
    AND hits.transaction.transactionRevenue IS NOT NULL
    AND hits_product.productSku IS NOT NULL
ORDER BY
    date
'''

job_config = bigquery.QueryJobConfig()
job_config.query_parameters = query_params
df = client.query(query, job_config=job_config).to_dataframe()

# get scale indexes
identifiers = schema[(schema['Status'] == 'SELECTED')
                     & (schema['Scale'] == 'IDENTIFIER')].index
identifiers = identifiers.union(pd.Index(['session_id']))

nominal = schema[(schema['Status'] == 'SELECTED')
                 & (schema['Scale'] == 'NOMINAL')].index

binary = schema[(schema['Status'] == 'SELECTED')
                & (schema['Scale'] == 'BINARY')].index

numeric = schema[(schema['Status'] == 'SELECTED')
                 & (schema['Scale'] == 'NUMERIC')].index

timestamp = schema[(schema['Status'] == 'SELECTED')
                   & (schema['Scale'] == 'TIMESTAMP')].index

# Cast variables to proper dtypes
df[identifiers] = df[identifiers].astype(str)
df[nominal] = df[nominal].astype('category')
df[timestamp] = pd.to_datetime(df[timestamp].squeeze())

# Prints dataframe charactersitics
with pd.option_context('display.max_columns', None):
    display(df.head())
    
display(df.info())
display(df.describe(include=np.number))

with pd.option_context('display.max_columns', None):
    display(df.describe(exclude=np.number))

### 5.3.2 Data Preparation

In [ ]:
# HANDLE MISSING VALUES
# ---------------------

# Plot missing values patterns
nan_chars = {'nan',
             'None',
             '(none)',
             '(not set)',
             'not available in demo dataset'}
             
nan_mask = (df.isna() 
            | df.isnull() 
            | (df == 'nan')
            | (df == 'None')
            | (df == '(none)') 
            | (df == '(not set)') 
            | (df == 'not available in demo dataset'))
plt.figure(figsize=(15, 15))
ax = sns.heatmap(nan_mask, cbar=False);
plt.title('Missing Data Patterns - White is missing')
plt.show()

In [ ]:
# Plot missing values proportion across variables and cases
nan_prop_vars = ((nan_mask.sum() / nan_mask.index.size)
                 .rename('proprotion of missing values in variables')
                 .sort_values(ascending=False))

nan_prop_cases = ((nan_mask.sum(axis=1) / nan_mask.columns.size)
                 .rename('proprotion of missing values in cases')
                 .sort_values(ascending=False))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
nan_prop_vars.hist(ax=ax1)
ax1.set_title('Proportion of missing values in variables')
nan_prop_cases.hist(ax=ax2)
ax2.set_title('Proprotion of missing values in cases');

In [ ]:
# Analyze proportion of missing values and missing vlaue codes in variables
def analyze_nan_levels(var, nan_chars):
    levels = var.unique()
    nan_levels = set(levels) & nan_chars | set(levels[pd.isna(levels)])
    return pd.Series({'num_levels': len(levels),
                      'levels': set(levels),
                      'num_nan_levels': len(nan_levels),
                      'nan_levels': nan_levels})

nan_vars_summary = nan_prop_vars.to_frame('nan_proportion')
nan_vars_summary[['num_levels', 
                  'levels', 
                  'num_nan_levels', 
                  'nan_levels']] = (df.apply(analyze_nan_levels, 
                                             args=(nan_chars,))
                                    .T)

display(nan_vars_summary[nan_prop_vars > 0])

In [ ]:
# FEATURE ENGINEERING
# -------------------
re_engineered_vars = ['traffic_keyword',
                      'product_brand',
                      'product_category',
                      'sales_region',
                      'avg_visits',
                      'avg_pageviews',
                      'avg_time_on_site',
                      'social_referral',
                      'ad_campaign',
                      'avg_product_revenue',
                      'unique_products']

# Reconstruct traffic variables
# -----------------------------
df['traffic_keyword'] = (df['traffic_keyword']
                         .astype(str)
                         .apply(helper.reconstruct_traffic_keyword)
                         .astype('category'))


# Engineer brand variable from brand keywords in product name
# and brand variables. Remaining (not set) brand is Google brand
# --------------------------------------------------------------
product_df = helper.query_product_info(client, query_params)
df['product_brand'] = (helper.reconstruct_brand(df['product_sku'], product_df)
                       .astype('category'))

# Engineer category labels from category variables and keywords in product name.
# Remaining (not set) brand is predicted by Naive Bayes Model with 
# f1 weighted test score > 0.8% (0.985)
# ------------------------------------------------------------------------------

# load mappings from category variables to category_label
product_category = (
    pd.read_excel('product_categories.xlsx', 
                  sheet_name='product_category_id')
    [['product_category', 'category_label']]
)

product_category_grp = pd.read_excel('product_categories.xlsx', 
                                        sheet_name='product_category_grp_id')

category_spec = {'product_category': product_category,
                 'product_category_grp': product_category_grp}

# reconstruct category labels from category variables and product names
df['product_category'] = (
    helper.reconstruct_category(df['product_sku'], product_df, category_spec)
    .astype('category')
)

# reconstruct sales region from subcontinent labels
# -------------------------------------------------
df['sales_region'] = (df['subcontinent']
                       .apply(helper.reconstruct_sales_region)
                       .astype('category'))

# cast binary variable is_social_referral
df['social_referral'] = ((df['is_social_referral']
                          .replace(['Yes', 'No'], [True, False]))
                          if df['is_social_referral'].dtype != 'bool'  
                          else df['is_social_referral'])

# create ad campaign flag from campaign and add variables
ad_vars = ['campaign', 'ad_content', 'ad_page', 'ad_slot', 'ad_network_type']
df['ad_campaign'] = False
df.loc[~nan_mask[ad_vars].all(axis=1), 'ad_campaign'] = True

In [ ]:
# drop cases and variables with high proportion of missing values
# ---------------------------------------------------------------

# Variables with high proportion of missing values
drop_vars = [
    'referral_path',                 # not valuable information
    'product_name',                  # too detailed, category sufficient
    'product_brand',                 # reconstructed
    'product_brand_grp',             # reconstructed
    'product_category_grp',          # reconstructed
    'is_social_referral',            # reconstructed social_referral
    'campaign',                      # reconstructed is_ad_campaign
    'ad_content',                    # reconstructed is_ad_campaign
    'ad_page',                       # reconstructed is_ad_campaign
    'ad_slot',                       # reconstructed is_ad_campaign
    'ad_network_type',               # reconstructed is_ad_campaign
    'domain',                        # high proportion of missing values
    'city',                          # high proportion of missing values
    'region',                        # high proportion of missing values
    'metro',                         # high proportion of missing values
    'direct_traffic',                # collinear with medium & channel group
    'medium',                        # collinear with channel group
    'social_action',                 # no valid values
    'social_interaction_network',    # no valide values
    'network_action',                # not valuable information
    'social_interactions',           # no valide values
    'unique_social_interactions'     # no valide values
    
]

# cases
cases = (nan_mask[['continent', 
                   'subcontinent', 
                   'country',
                   'time_on_site']]
         .any(axis=1))

# delete variables and cases
clean_df = (df.drop(columns=drop_vars, errors='ignore')
              .drop(index=df.index[cases], errors='ignore'))

# check for missing values
clean_mask = (clean_df.isna() 
                | clean_df.isnull() 
                | (clean_df == 'nan')
                | (clean_df == 'None')
                | (clean_df == '(none)') 
                | (clean_df == '(not set)') 
                | (clean_df == 'not available in demo dataset'))

# encode and aggregate engineered variables on customer level
# -----------------------------------------------------------
agg_df = helper.aggregate_data(clean_df)

# dipslay summary
txt = 'MISSING VALUE AND RE-EGINEERING SUMMARY'
print(txt + '\n' + '-'*len(txt) + '\n' )
txt = ('Missing values: {} ({:.1f}%)'
       .format(nan_mask.sum().sum(),
               nan_mask.sum().sum() / np.product(nan_mask.shape) * 100))       
print(txt + '\n')

txt = ('Re-engineered variables: {} ({:.1f}%)'
       .format(len(re_engineered_vars), 
               len(re_engineered_vars) / df.columns.size * 100))   
print(txt)
print(re_engineered_vars, '\n')

txt = ('Deleteted variables: {} ({:.1f}%)'
       .format(len(drop_vars), 
               len(drop_vars) / df.columns.size * 100))       
print(txt)
print(drop_vars, '\n')

txt = ('Deleteted cases: {} ({:.1f}%)'
       .format(cases.sum(), cases.sum() / df.index.size * 100))       
print(txt)
display(df[cases])

txt = ('\nMissing values: {} ({:.1f}%)'
       .format(clean_df.isna().sum().sum(),
               clean_df.isna().sum().sum() / np.prod(clean_df.shape)))       
print(txt + '\n')

txt = ('ENCODED AND AGGREGATED DATA ON CLIENT LEVEL:')
print(txt, '\n' + '-' * len(txt))
display(agg_df)

In [ ]:
# add buyer personas (customer clusters) to additional data
data = pd.merge(agg_df, 
                X[X.columns[X.columns.str.contains('cluster')]],
                how='left',
                left_index=True,
                right_index=True)

## 6. Deployment